# Model Role Prediction For Legend Stats

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


## Define Legend Categories

In [ ]:
# Define legend categories
legend_categories = {
    'Assault': ['Bangalore', 'Fuse', 'Ash', 'Mad Maggie', 'Ballistic'],
    'Skirmisher': ['Pathfinder', 'Wraith', 'Octane', 'Revenant', 'Horizon', 'Valkyrie', 'Alter'],
    'Recon': ['Bloodhound', 'Crypto', 'Seer', 'Vantage'],
    'Support': ['Gibraltar', 'Lifeline', 'Mirage', 'Loba', 'Newcastle', 'Conduit'],
    'Controller': ['Caustic', 'Wattson', 'Rampart', 'Catalyst']
}
legend_to_category = {legend: role for role, legends in legend_categories.items() for legend in legends}
legends = list(legend_to_category.keys())

In [ ]:


base_path = r""
damage_path = os.path.join(base_path, 'Legend Damage')
kills_path = os.path.join(base_path, 'Legend Kills')

all_data = []
for legend in legends:
    try:
        legend_file = legend
        kills_file = os.path.join(kills_path, f"{legend_file}_kills.csv")
        damage_file = os.path.join(damage_path, f"{legend_file}_damage.csv")
        # Read kills and damage WITH HEADER
        kills = pd.read_csv(kills_file)
        damage = pd.read_csv(damage_file)
        # Use last column if unsure of name
        kills_col = kills.columns[-1]
        damage_col = damage.columns[-1]
        # Clean up values (remove commas, convert to int)
        kills[kills_col] = kills[kills_col].astype(str).str.replace(',', '').astype(int)
        damage[damage_col] = damage[damage_col].astype(str).str.replace(',', '').astype(int)
        n = min(len(kills), len(damage))
        df = pd.DataFrame({
            'Kills': kills[kills_col].values[:n],
            'Damage': damage[damage_col].values[:n],
            'legend': legend,
            'role': legend_to_category[legend]
        })
        all_data.append(df)
        #print(f"Loaded: {legend} ({n} samples)")
    except Exception as e:
        print(f"Skipping {legend} due to missing file: {e}")

df_all = pd.concat(all_data, ignore_index=True)
# Model
features = ['Kills', 'Damage']
X = df_all[features]
y = df_all['role']

# No Stratified split, just a random split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Stratified split to maintain class distribution
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))